# Part 1A: Data Collection Spotify (Audio Features)

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import shutil

## Mengatur API dari Spotify
Untuk scrape data, kita akan menggunakan API wrapper untuk Spotify, Spotipy

### Mengambil Data Lagu

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= '3ff80dfa932f42ccba937689112ebaf8',
                                                           client_secret= 'b222a7c17fda476aa56057b10f263a7d'),
                    requests_timeout=30)

In [33]:
def get_playlist_tracks(playlist_id, n_tracks):
    data = pd.DataFrame()
    #spotify API sets a default limit of 100 tracks, to get all we need to invoke offset
    for i in range(0,n_tracks,100):
        results = sp.playlist_tracks(playlist_id,limit=100, offset=i)
        #shape json into dataframe
        track_ids = [track.get('track').get('id') for track in results.get('items')]
        df = pd.DataFrame()
        df['id'] = track_ids
        df['track'] = [track.get('track').get('name') for track in results.get('items')]
        df['artist_id'] = [track.get('track').get('artists')[0].get('id') for track in results.get('items')]
        df['artist'] = [track.get('track').get('artists')[0].get('name') for track in results.get('items')]
        df['album_id'] = [track.get('track').get('album').get('id') for track in results.get('items')]
        df['album'] = [track.get('track').get('album').get('name') for track in results.get('items')]
        df['release_date'] = [track.get('track').get('album').get('release_date') for track in results.get('items')]
        df['link'] = [track.get('track').get('href') for track in results.get('items')]
        #get audio features, merge with df
        track_features = sp.audio_features(track_ids)
        df_features = pd.DataFrame.from_dict(track_features)
        df = df.merge(df_features, on='id', how='left')
        #merge with data
        data = pd.concat([data,df])
    return data

In [34]:
top50 = get_playlist_tracks('37i9dQZEVXbObFQZ3JLcXt', 50)

TypeError: 'NoneType' object is not subscriptable

In [7]:
top50.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,26cvTWJq2E1QqN4jyH2OTU,Tak Segampang Itu,1JvbNeV9zG9Sew1JyaWsyx,Anggi Marito,6gO5mDyNTqiIqHItRil4OG,Tak Segampang Itu,2022-12-02,https://api.spotify.com/v1/tracks/26cvTWJq2E1Q...,0.506,0.438,...,0.000009,0.109,0.178,129.538,audio_features,spotify:track:26cvTWJq2E1QqN4jyH2OTU,https://api.spotify.com/v1/tracks/26cvTWJq2E1Q...,https://api.spotify.com/v1/audio-analysis/26cv...,231100,4
1,2AaaE0qvFWtyT8srKNfRhH,Komang,5LcNPa8f8bRmZqELgoRFkN,Raim Laode,2N8JUijzZsT2IJnplY4vAE,Komang,2022-08-17,https://api.spotify.com/v1/tracks/2AaaE0qvFWty...,0.696,0.412,...,0.000062,0.100,0.349,133.913,audio_features,spotify:track:2AaaE0qvFWtyT8srKNfRhH,https://api.spotify.com/v1/tracks/2AaaE0qvFWty...,https://api.spotify.com/v1/audio-analysis/2Aaa...,222707,4
2,6dXiWwFrcGieqnoLYzPNp5,Sial,3wOsYKZM0zcKNasi3I7fP4,Mahalini,6iBh7T1cUR8MPrtly5xugU,fábula,2023-01-23,https://api.spotify.com/v1/tracks/6dXiWwFrcGie...,0.563,0.431,...,0.000003,0.123,0.196,120.014,audio_features,spotify:track:6dXiWwFrcGieqnoLYzPNp5,https://api.spotify.com/v1/tracks/6dXiWwFrcGie...,https://api.spotify.com/v1/audio-analysis/6dXi...,243725,4
3,1daDRI9ahBonbWD8YcxOIB,Angels Like You,5YGY8feqx7naU7z4HrwZM6,Miley Cyrus,5BRhg6NSEZOj0BR6Iz56fR,Plastic Hearts,2020-11-27,https://api.spotify.com/v1/tracks/1daDRI9ahBon...,0.672,0.642,...,0.000000,0.100,0.494,121.981,audio_features,spotify:track:1daDRI9ahBonbWD8YcxOIB,https://api.spotify.com/v1/tracks/1daDRI9ahBon...,https://api.spotify.com/v1/audio-analysis/1daD...,196453,4
4,5BKJAuVMDyrFvUTo4dIaiK,Not You,7vk5e3vY1uw9plTHJAMwjN,Alan Walker,3KrkQ77DF9OUB0aOzKFYOF,World Of Walker,2021-11-26,https://api.spotify.com/v1/tracks/5BKJAuVMDyrF...,0.573,0.264,...,0.000017,0.103,0.379,129.791,audio_features,spotify:track:5BKJAuVMDyrFvUTo4dIaiK,https://api.spotify.com/v1/tracks/5BKJAuVMDyrF...,https://api.spotify.com/v1/audio-analysis/5BKJ...,153719,4


In [9]:
top50.to_csv('datasets/audio_features.csv')

### Mengambil Data Artis

In [35]:
def get_artist_details(artist_id):
    d = {}
    results = sp.artist(artist_id)
    d['artist_id'] = artist_id
    d['artist'] = results.get('name')
    if len(results.get('images'))>0:
        d['image'] = results.get('images')[0].get('url')
    else:
        d['image'] = ''
    d['genres'] = str(results.get('genres'))
    d['popularity'] = results.get('popularity')
    d['link'] = results.get('href')
    if len(results.get('followers'))>0:
        d['followers'] = results.get('followers').get('total')
    else:
        d['followers'] = None
    return d

In [36]:
artists = list(set([artist for artist in top50['artist_id']]))

In [37]:
artist_data = []
for artist in artists:
    data = get_artist_details(artist)
    artist_data.append(data)

IndexError: list index out of range

In [14]:
df_artists = pd.DataFrame(artist_data)
df_artists.head()

,artist_id,artist,image,genres,popularity,link,followers
0,7Ln5yumFjHCkeZ8bAzHUcp,Yovie Widianto,https://i.scdn.co/image/ab6761610000e5eb5146b8...,['indonesian pop'],64,https://api.spotify.com/v1/artists/7Ln5yumFjHC...,198778
1,6FTLayBxjkQeanFdUusk1I,Fabio Asher,https://i.scdn.co/image/ab6761610000e5ebd7384b...,['indonesian pop'],65,https://api.spotify.com/v1/artists/6FTLayBxjkQ...,736372
2,4AK6F7OLvEQ5QYCBNiQWHq,One Direction,https://i.scdn.co/image/5bb443424a1ad71603c43d...,"['boy band', 'pop', 'post-teen pop', 'talent s...",85,https://api.spotify.com/v1/artists/4AK6F7OLvEQ...,31158728
3,0Pk4JEXgC64RBmovnQDZ27,Jogja Hip Hop Foundation,https://i.scdn.co/image/ab67616d0000b273de748f...,"['indonesian hip hop', 'lagu jawa']",62,https://api.spotify.com/v1/artists/0Pk4JEXgC64...,623252
4,3wOsYKZM0zcKNasi3I7fP4,Mahalini,https://i.scdn.co/image/ab6761610000e5ebb83334...,['indonesian pop'],75,https://api.spotify.com/v1/artists/3wOsYKZM0zc...,5636815


In [15]:
df_artists.to_csv('datasets/artists.csv')

### Mengunduh Foto Artis

In [16]:
#Code taken from - https://towardsdatascience.com/how-to-download-an-image-using-python-38a75cfa21c    
def get_image(image_url, filename):
    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)
       # print('Image sucessfully Downloaded: ',filename)
    else:
        print('Image Couldn\'t be retreived', filename)

In [17]:
df_artists.set_index('artist_id',inplace=True)

In [20]:
for index in df_artists.index:
    artist_id = index
    link = df_artists.loc[index]['image']
    if 'http' in link:
        path = 'images/artists/' + artist_id + '.png'
        get_image(link, path)

In [21]:
#taken from StackOverflow https://stackoverflow.com/questions/51486297/cropping-an-image-in-a-circular-way-using-python
import numpy as np
from PIL import Image, ImageDraw


def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

def crop_image(image_path, new_path):
    img=Image.open(image_path).convert("RGB")
    img = crop_max_square(img)
    npImage=np.array(img)
    h,w=img.size
    alpha = Image.new('L', img.size,0)
    draw = ImageDraw.Draw(alpha)
    draw.pieslice([0,0,h,w],0,360,fill=255)
    npAlpha=np.array(alpha)
    npImage=np.dstack((npImage,npAlpha))
    Image.fromarray(npImage).save(new_path)

In [22]:
import os 
files = os.listdir('images/artists')
file_paths = ['images/artists/' + file for file in files]

for path in file_paths:
    new_path = 'images/cropped_artists/' + path[18:]
    crop_image(path, new_path)